In [ ]:
! pip install textattack  sentence_transformers evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 418 kB 35.3 MB/s 
     |████████████████████████████████| 85 kB 2.3 MB/s 
     |████████████████████████████████| 72 kB 745 kB/s 
     |████████████████████████████████| 365 kB 72.6 MB/s 
     |████████████████████████████████| 125 kB 79.0 MB/s 
     |████████████████████████████████| 5.8 MB 73.1 MB/s 
     |████████████████████████████████| 41.4 MB 206 kB/s 
     |████████████████████████████████| 60 kB 8.4 MB/s 
     |████████████████████████████████| 3.6 MB 73.3 MB/s 
     |████████████████████████████████| 401 kB 80.3 MB/s 
     |████████████████████████████████| 769 kB 88.2 MB/s 
     |████████████████████████████████| 182 kB 97.3 MB/s 
     |████████████████████████████████| 212 kB 91.9 MB/s 
     |████████████████████████████████| 132 kB 105.7 MB/s 
     |████████████████████████████████| 95 kB 5.5 MB/s 
     |████████████████████████████████

In [ ]:
import textattack
import pandas as pd
import json

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset
from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker
from textattack import AttackArgs
from textattack.datasets import Dataset

# Import the model
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset

In [ ]:
root = '/content/drive/Shareddrives/BipinA/EE8227_secure_ml/Project'

In [ ]:
from datasets import load_from_disk

In [ ]:
dataset = load_from_disk(f"/content/drive/Shareddrives/BipinA/EE8227_secure_ml/Project/data/imdb_test_6k.hf")

In [ ]:
test_dataset = HuggingFaceDataset(dataset)

In [ ]:
len(test_dataset)

6000

# Adversarially Train bert-base-uncased

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import torch

In [ ]:
import os 

In [ ]:
torch.cuda.device_count()

1

In [ ]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

In [ ]:
from textattack.attack_recipes import A2TYoo2021

In [ ]:
from datasets import load_dataset, load_from_disk, concatenate_datasets

In [ ]:
root= "/content/drive/Shareddrives/BipinA/EE8227_secure_ml/Project"

In [ ]:
dataset_train = load_dataset("imdb", split="train")
dataset_test = load_dataset("imdb", split="test")

dataset_train.save_to_disk(f"{root}/data/imdb_train.hf")
dataset_test.save_to_disk(f"{root}/data/imdb_test.hf")

In [ ]:
dataset_train = load_from_disk(f"{root}/data/imdb_train.hf")
dataset_test = load_from_disk(f"{root}/data/imdb_test.hf")

def split_dataset(ds):
  ds_1 = ds.filter(lambda x: x["label"] == 1).shuffle(seed=42).select(range(3000))
  ds_0 = ds.filter(lambda x: x["label"] == 0).shuffle(seed=42).select(range(3000))
  return concatenate_datasets([ds_1,ds_0])

ds_test_6000 = split_dataset(dataset_test)
ds_train_6000 = split_dataset(dataset_train)

ds_test_6000.save_to_disk(f"{root}/data/imdb_test_6k.hf")
ds_train_6000.save_to_disk(f"{root}/data/imdb_train_6k.hf")

In [ ]:
test_ds = load_from_disk(f"{root}/data/imdb_test_6k.hf")
train_ds = load_from_disk(f"{root}/data/imdb_train_6k.hf")

In [ ]:
train_dataset = HuggingFaceDataset(train_ds)
eval_dataset = HuggingFaceDataset(test_ds)

In [ ]:
# We only use A2TYoo2021_1 to demonstration purposes.
attack = A2TYoo2021.build(model_wrapper)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
bool(torch.cuda.device_count())

True

In [ ]:
# Train for 3 epochs with 1 initial clean epochs, 1000 adversarial examples per epoch, learning rate of 5e-5, and effective batch size of 32 (8x4).
training_args = textattack.TrainingArgs(
    num_epochs=3,
    num_clean_epochs=1,
    num_train_adv_examples=0.2,
    learning_rate=5e-5,
    early_stopping_epochs = 3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    output_dir=f"{root}/adv_trained_model",
    save_last= True,
    checkpoint_interval_epochs = 2,
    parallel = bool(torch.cuda.device_count())
)

In [ ]:
trainer = textattack.Trainer(
    model_wrapper,
    "classification",
    attack,
    train_dataset,
    eval_dataset,
    training_args
)
trainer.train()

textattack: Writing logs to /content/drive/Shareddrives/BipinA/EE8228neural_info/Project/code/output_checkpoints/train_log.txt.
textattack: Wrote original training args to /content/drive/Shareddrives/BipinA/EE8228neural_info/Project/code/output_checkpoints/training_args.json.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
textattack: ***** Running training *****
textattack:   Num examples = 6000
textattack:   Num epochs = 3
textattack:   Num clean epochs = 1
textattack:   Instantaneous batch size per device = 8
textattack:   Total train batch size (w. parallel, distributed & accumulation) = 32
textattack:   Gradient accumulation steps = 4
textattack:   Total optimization steps = 638
textattack: ===================================

Loss 0.36165: 100%|██████████| 900/900 [10:58<00:00,  1.37it/s]
textattack: Train accuracy: 90.11%
textattack: Eval accuracy: 92.87%
textattack: Best score found. Saved model to /content/drive/Shareddrives/BipinA/EE8228neural_info/Project/code/output_checkpoints/best_model/
textattack: ==========================================================
textattack: Epoch 3
textattack: Attacking model to generate new adversarial training set...
[Succeeded / Failed / Skipped / Total] 1200 / 1606 / 75 / 2881: 100%|██████████| 1200/1200 [6:47:08<00:00, 20.36s/it]


textattack: Total number of attack results: 2881
textattack: Attack success rate: 42.77% [1200 / 2806]


Loss 0.27981: 100%|██████████| 900/900 [10:59<00:00,  1.36it/s]
textattack: Train accuracy: 95.11%
textattack: Eval accuracy: 93.33%
textattack: Best score found. Saved model to /content/drive/Shareddrives/BipinA/EE8228neural_info/Project/code/output_checkpoints/best_model/
textattack: Wrote README to /content/drive/Shareddrives/BipinA/EE8228neural_info/Project/code/output_checkpoints/README.md.


In [ ]:
trainer.model_wrapper.model